# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
weather_df = pd.read_csv("../output/cities.csv")
weather_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Yambio,SS,4.5721,28.3955,73.08,77,73,3.11
1,1,Yellowknife,CA,62.4560,-114.3525,26.15,80,20,2.30
2,2,Bluff,NZ,-46.6000,168.3333,52.00,90,88,7.00
3,3,Cape Town,ZA,-33.9258,18.4232,68.05,49,0,6.91
4,4,Port Elizabeth,ZA,-33.9180,25.5701,66.20,72,0,14.97
...,...,...,...,...,...,...,...,...,...
565,565,Daura,NG,11.5538,11.4085,95.25,7,94,3.94
566,566,Salinas,US,36.6777,-121.6555,52.97,76,90,3.44
567,567,Burla,IN,21.5000,83.8667,85.87,24,12,3.06
568,568,Moyale,KE,3.5167,39.0584,71.26,73,60,3.67


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#Configure gamps with API key
gmaps.configure(api_key=g_key)

#Use Lat and Lng as location
location = weather_df[["Lat","Lng"]].astype(float)

#Use humidity as weight
humidity = weather_df["Humidity"].astype(float)

In [13]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights = humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#create a new DF matching given weather critera
#max temp between 70 and 80 degrees
#wind speed less than 10 mph
#zero cloudiness
#drop all rows that don't match above conditions

perfect_cities_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) &
                                  (weather_df["Wind Speed"] < 10 ) & (weather_df["Cloudiness"] == 0)]

perfect_cities_df.dropna()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
159,159,Pisco,PE,-13.7000,-76.2167,71.35,73,0,9.22
255,255,Akdepe,TM,42.0551,59.3788,70.72,24,0,4.61
357,357,Catalina Foothills,US,32.2979,-110.9187,71.42,14,0,2.80
493,493,Santa Rosalía,MX,27.3167,-112.2833,73.89,22,0,4.05
506,506,Bushehr,IR,28.9684,50.8385,71.60,78,0,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
#create df for finding hotels
hotel_df = perfect_cities_df.loc[:,["City","Country", "Lat", "Lng"]]
#add column for hotel name
hotel_df["Hotel Name "] =""
hotel_df

#set base url & parameters for hotel search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
         "keyword": "hotel",
         "radius" : 5000,
         "key":g_key}

print("-------------------------------------------------------------------")
print("Start of Search")
print("-------------------------------------------------------------------")
#use for loop to get lat and long for each city and insert into params, once we have full params use same for loop w/ try except to print out search reults
for index, row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    print(f"Searching for Hotels in City #{index}: {city}")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"The closest hotel for {city} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except(KeyError, IndexError):
        print(f"No hotel results found, Skipping..........")
print("-------------------------------------------------------------------")
print("End of Search")
print("-------------------------------------------------------------------")

-------------------------------------------------------------------
Start of Search
-------------------------------------------------------------------
Searching for Hotels in City #159: Pisco
The closest hotel for Pisco is Hostal Tambo Colorado
Searching for Hotels in City #255: Akdepe
No hotel results found, Skipping..........
Searching for Hotels in City #357: Catalina Foothills
The closest hotel for Catalina Foothills is Westward Look Wyndham Grand Resort and Spa
Searching for Hotels in City #493: Santa Rosalía
The closest hotel for Santa Rosalía is Hotel Las Casitas Santa Rosalia
Searching for Hotels in City #506: Bushehr
The closest hotel for Bushehr is Sky 2 hotel
-------------------------------------------------------------------
End of Search
-------------------------------------------------------------------


In [20]:
hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel Name
159,Pisco,PE,-13.7000,-76.2167,,Hostal Tambo Colorado
255,Akdepe,TM,42.0551,59.3788,,NaN
357,Catalina Foothills,US,32.2979,-110.9187,,Westward Look Wyndham Grand Resort and Spa
493,Santa Rosalía,MX,27.3167,-112.2833,,Hotel Las Casitas Santa Rosalia
506,Bushehr,IR,28.9684,50.8385,,Sky 2 hotel


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [22]:
# Add marker layer ontop of heat map
pins = gmaps.marker_layer(locations, info_box_content =hotel_info)

fig.add_layer(pins)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))